In [3]:
import pandas as pd
import os
import pandas as pd

datos_path = '/Users/tomasvazquez/Desktop/Facultad/Tesis/Datos/'

### Para generar el csv con los 3 años concatenados

In [38]:
GEN = pd.read_excel(datos_path+'GEN.xlsx', sheet_name='2021')

In [39]:
GEN2 = pd.read_excel(datos_path+'GEN.xlsx', sheet_name='2022')
GEN3 = pd.read_excel(datos_path+'GEN.xlsx', sheet_name='2023')

In [40]:
df_concatenado = pd.concat([GEN, GEN2, GEN3])

In [41]:
df_concatenado.to_csv(datos_path + 'GEN_total.csv', index=False)

In [42]:
GEN_loaded = pd.read_csv(datos_path+'GEN_total.csv')

/var/folders/pq/p8msjkrx53q8b1ts29trjghw0000gn/T/ipykernel_93294/81847354.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

In [43]:
GEN_loaded['Fecha']

0         2021-07-01 00:00:00
1         2021-07-01 00:10:00
2         2021-07-01 00:20:00
3         2021-07-01 00:30:00
4         2021-07-01 00:40:00
                 ...         
135934    2024-01-30 23:20:00
135935    2024-01-30 23:30:00
135936    2024-01-30 23:40:00
135937    2024-01-30 23:50:00
135938    2024-01-31 00:00:00
Name: Fecha, Length: 135939, dtype: object

### Funcion para levantar las columnas con los nodos que querramos, y guardar en un csv con los nombres de los buses

In [46]:
def create_dataset(path, buses, name=None):
    original_data = pd.read_csv(path)
    clean_data = pd.DataFrame(columns=buses.keys())
    clean_data['Fecha'] = pd.to_datetime(clean_data['Fecha'])
    for bus in buses:
        if bus != 'Fecha':
            clean_data[bus] = original_data[bus]
            clean_data = clean_data.rename(columns={bus: buses[bus]})
            clean_data = clean_data.applymap(convertir_a_cero)
    if 'Fecha' in buses:
        clean_data['Fecha'] = original_data['Fecha']
    if name:
        clean_data.to_csv(datos_path + name, index=False)

    return clean_data

def convertir_a_cero(elemento):
    if isinstance(elemento, str):
        return 0
    else:
        return elemento

In [51]:
buses_pload = {'Fecha': 'Fecha',
'SGA500.013 GE G01 MED P.AV': '4000_1',
'SGA500.013 GE G02 MED P.AV': '4000_2',
'SGA500.013 GE G03 MED P.AV': '4000_3',
'SGA500.013 GE G04 MED P.AV': '4000_4',
'SGA500.013 GE G05 MED P.AV': '4000_5',
'SGA500.013 GE G06 MED P.AV': '4000_6',
'SGA500.013 GE G13 MED P.AV': '4000_7',
'SGU500.013 GE G07 MED P.AV': '90000_1',
'SGU500.013 GE G08 MED P.AV': '90000_2',
'SGU500.013 GE G09 MED P.AV': '90000_3',
'SGU500.013 GE G10 MED P.AV': '90000_4',
'SGU500.013 GE G11 MED P.AV': '90000_5',
'SGU500.013 GE G12 MED P.AV': '90000_6',
'SGU500.013 GE G14 MED P.AV': '90000_7',
'PTIGRE.011 GE G01 MED P.AV': '90160_1',
'PTIGRE.011 GE G02 MED P.AV': '90160_2',
'PTIGRE.011 GE G03 MED P.AV': '90160_3',
'PTIGRE.011 GE G04 MED P.AV': '90160_4',
'PTIGRE.011 GE G05 MED P.AV': '90160_5',
'PTIGRE.011 GE G06 MED P.AV': '90160_6',
'PTIGRE.150 GE PTI APRC MED P.AV': '90160_7',
'PTIGREB.150 GE STG MED P.AV': '90160_8',
'PTIGRECC.150 GE GST MED P.AV':'90160_9' ,
'PTIGRECC.150 GE GT1 MED P.AV':'90160_10' ,
'PTIGRECC.150 GE GT2 MED P.AV':'90160_11' ,
'EKIYU.030 GE TOT MED P.AV': '90160_12',
'CENPAL.500 GE G01 MED P.AV': '90500_1',
'CENPAL.500 GE G02 MED P.AV': '90500_2',
'CENPAL.500 GE G03 MED P.AV': '90500_3',
'NARANJAL.150 GE TOT MED P.AV':'92000' ,
'DLITORAL.030 GE TOT MED P.AV':'92040' ,
'PALOMAS.030 GE TOT MED P.AV': '92061',
'JPTERRA.030 GE TOT MED P.AV': '92091',
'MVL150.150 GE CT1 MED P.AV': '92124_1',
'MVL150.150 GE CT2 MED P.AV': '92124_2',
'CBMOT.011 GE G01 MED P.AV': '92240_1',
'CBMOT.011 GE G02 MED P.AV': '92240_2',
'CBMOT.011 GE G03 MED P.AV': '92240_3',
'CBMOT.011 GE G04 MED P.AV': '92240_4',
'CBMOT.011 GE G05 MED P.AV': '92240_5',
'CBMOT.011 GE G06 MED P.AV': '92240_6',
'CBMOT.011 GE G07 MED P.AV': '92240_7',
'CBMOT.011 GE G08 MED P.AV': '92240_8',
'BIOENER.030 GE TOT MED P.AV': '92400_1',
'PONLAR.030 GE TOT MED P.AV': '92400_2',
'EPAMPA.030 GE TOT MED P.AV': '92411',
'EVALENTI.030 GE TOT MED P.AV':'92420' ,
'PCGRANDE.030 GE TOT MED P.AV':'92485' ,
'MELOWIND.030 GE TOT MED P.AV':'92490' ,
'EFLOR1.030 GE TOT MED P.AV': '92524_1',
'EFLOR2.030 GE TOT MED P.AV': '92524_2',
'BONETE.013 GE TOT MED P.AV': '92540',
'BAYGORRI.150 GE G01 MED P.AV':'92550_1' ,
'BAYGORRI.150 GE G02 MED P.AV':'92550_2' ,
'BAYGORRI.150 GE G03 MED P.AV':'92550_3' ,
'TMACIEL1.030 GE TOT MED P.AV':'92590_1' ,
'TMACIEL2.030 GE TOT MED P.AV':'92590_2' ,
'ECARIAS.030 GE TOT MED P.AV': '92590_3',
'ENPASTO1.030 GE TOT MED P.AV':'92590_4' ,
'ARTILLER.150 GE TOT MED P.AV':'92611' ,
'RDELSUR.030 GE TOT MED P.AV': '92700_1',
'EMALD2.030 GE TOT MED P.AV': '92700_2',
'MALDONAD.030 GE G01 MED P.AV':'92700_3' ,
'CARACOLE.030 GE TOT MED P.AV':'92700_4' ,
'EMINAS1.030 GE TOT MED P.AV': '92710',
'ECARAPE1.030 GE TOT MED P.AV':'92760_1' ,
'ECARAPE2.030 GE TOT MED P.AV':'92760_2' ,
'PERALTA1.030 GE TOT MED P.AV':'92790_1' ,
'PERALTA2.030 GE TOT MED P.AV':'92790_2' ,
'PALMATIR.030 GE TOT MED P.AV':'92790_3' ,
'LUZLOMA.030 GE TOT MED P.AV': '92910_1',
'LUZMAR.030 GE TOT MED P.AV': '92910_2',
'LUZRIO.030 GE TOT MED P.AV': '92910_3'}

In [52]:
create_dataset(datos_path+'GEN_total.csv', buses_pload)

/var/folders/pq/p8msjkrx53q8b1ts29trjghw0000gn/T/ipykernel_93294/360215318.py:9: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,

,Fecha,4000_1,4000_2,4000_3,4000_4,4000_5,4000_6,4000_7,90000_1,90000_2,...,92700_4,92710,92760_1,92760_2,92790_1,92790_2,92790_3,92910_1,92910_2,92910_3
0,2021-07-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000,0.000,0.0000,0.0000,0.000,0.0,0.0,0.0
1,2021-07-01 00:10:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000,0.000,0.0000,0.0000,0.000,0.0,0.0,0.0
2,2021-07-01 00:20:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000,0.000,0.0000,0.0000,0.000,0.0,0.0,0.0
3,2021-07-01 00:30:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000,0.000,0.0000,0.0000,0.000,0.0,0.0,0.0
4,2021-07-01 00:40:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000,0.000000,0.000,0.000,0.0000,0.0000,0.000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135934,2024-01-30 23:20:00,130.000000,130.800003,130.600006,130.600006,129.199997,130.900009,129.600006,127.900002,128.000000,...,4.723,26.436001,9.480,9.264,1.9845,0.0000,1.944,2.3,1.7,4.6
135935,2024-01-30 23:30:00,129.300003,131.900009,130.300003,130.199997,129.400009,130.699997,129.600006,128.600006,127.900002,...,2.815,28.188000,10.728,9.828,1.9845,0.0000,1.392,2.3,1.7,4.0
135936,2024-01-30 23:40:00,128.900009,132.300003,130.000000,130.199997,129.100006,130.000000,129.600006,128.699997,128.199997,...,1.653,26.382000,9.828,11.226,1.9845,0.0000,1.392,2.3,1.1,3.3
135937,2024-01-30 23:50:00,129.500000,131.000000,130.800003,130.199997,129.300003,130.800003,129.600006,127.900002,128.199997,...,2.760,28.278000,11.292,11.160,2.6460,1.9845,1.908,2.3,1.1,3.3
